# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
import json
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
MODEL_SEARCH = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = "You are an AI assistant."

In [4]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

In [5]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [ ]:
def create_spotify_songs(prompt):
    system_prompt = "You are an AI assisstant helping with creating engaging spotify playlists."
    prompt +="""\
    Answer in a json format, containing a title for the playlist and a list of the songs with song name and artist/artists. See the following example:\
    {"name": "Example Playlist", "songs": [{"artist": "linkin park", "name": "emptiness machine"}, "{"artist": "breaking benjamin", "name": "diary of jane"}]}\
    Make the playlist 10 to 20 songs long.
    """
    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": prompt}]
    response = openai.chat.completions.create(model=MODEL_SEARCH, messages=messages, response_format={"type": "json_object"})
    return response

In [32]:
spotify_song_list_creator = {
    "type": "function",
    "name": "create_spotify_playlist",
    "description": "Creates a spotify playlist based on a description.",
    "parameters": {
        "type": "object",
        "properties": {
            "description_playlist": {
                "type": "string",
                "description": """A description of the desired playlist.The description should contain an overall theme. It can additionally create songs or artists as inspiration.\
                Example: Search for songs that fit the Theme 'a nice autumn day'. Use the song 'California Dreaming' as inspiration. Use artists similar to 'Green Day' as inspiration.""",
            },
        },
        "required": ["description_playlist"],
        "additionalProperties": False
    }
}

In [35]:
tools = [{"type": "function", "function": spotify_song_list_creator}]
CONST_RESPONSE = []

def handle_tools(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    response_playlist = create_spotify_songs(arguments.get("description_playlist"))
    return response_playlist


def chat_with_tool(message, history):
    history.append({"role": "user", "content": message})
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    response_tool = None
    if response.choices[0].finish_reason=="tool_calls":
        tool_call = response.choices[0].message.tool_calls[0]
        response_tool=handle_tools(response.choices[0].message)
        history.append({
            "role": "assistant",
            "content": "",
            "tool_calls": [
                {
                    "id": tool_call.id,
                    "function": {
                        "name": tool_call.function.name,
                        "arguments": tool_call.function.arguments
                    },
                    "type": tool_call.type
                }
            ]
        })
        history.append({"role": "assistant", "content": "","type": "function_call_output", "call_id": tool_call.id, "output": response_tool.choices[0].message.content})

    response_caught = (response_tool or response).choices[0].message.content
    history.append({"role":"assistant", "content":response_caught})
    print(history)
    return "", history

In [36]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")


    entry.submit(chat_with_tool, inputs=[entry, chatbot], outputs=[entry, chatbot])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


[{'role': 'user', 'content': 'PLease create a playlist with topic false hope'}, {'role': 'assistant', 'content': '', 'tool_calls': [{'id': 'call_0hjeSVWhVZ2Vj4IenG6nSOB5', 'function': {'name': 'create_spotify_playlist', 'arguments': '{"description_playlist":"Create a Spotify playlist with the theme of \'false hope\'. Include songs that embody feelings of longing, disillusionment, and the bittersweet nature of hopes that never materialize. Look for inspiration from artists like Radiohead and The National."}'}, 'type': 'function'}]}, {'role': 'assistant', 'content': '', 'type': 'function_call_output', 'call_id': 'call_0hjeSVWhVZ2Vj4IenG6nSOB5', 'output': 'Sure! Here\'s a playlist titled **"False Hope,"** featuring songs that embody longing, disillusionment, and the bittersweet nature of unfulfilled hopes. The selection includes tracks that resonate with the themes you\'re looking for, inspired by artists like Radiohead and The National:\n\n1. **"Street Spirit (Fade Out)" - Radiohead**\n2

In [11]:
resp = CONST_RESPONSE[0]
resp.__dict__

IndexError: list index out of range

In [12]:
messages = [{"role": "system", "content": system_message},
            {"role": "user", "content": "Please create a spotify playlist with theme of your own choice."}]
response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
response

ChatCompletion(id='chatcmpl-CRNIbe06josY7i1vPPLKY7zry65ps', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_i8nDLox2SMnNRmaiTJcbzJ0d', function=Function(arguments='{"description_playlist":"A lively summer beach party vibe. Use songs that create a fun atmosphere, perfect for dancing, socializing, and enjoying the sun. Include songs like \'Happy\' by Pharrell Williams and artists similar to \'Calvin Harris\' and \'Dua Lipa\'."}', name='create_spotify_playlist'), type='function')]))], created=1760640453, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=70, prompt_tokens=130, total_tokens=200, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_token

In [ ]:
message = response.choices[0].message
tool_call = message.tool_calls[0]
arguments = json.loads(tool_call.function.arguments)
response_playlist = create_spotify_songs(arguments.get("description_playlist"))
response_playlist

ChatCompletion(id='chatcmpl-CRNKCDtQwAZqUT1IBG4zUAgTjQgCo', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Here\'s a fun summer beach party playlist that will create an upbeat and lively atmosphere, perfect for dancing and socializing. I\'ve included songs similar to "Happy" by Pharrell Williams, as well as tracks from artists like Calvin Harris and Dua Lipa.\n\n### Summer Beach Party Playlist\n\n1. **Happy** - Pharrell Williams\n2. **One Kiss** - Calvin Harris & Dua Lipa\n3. **Levitating** - Dua Lipa (feat. DaBaby)\n4. **Feel So Good** - Mase\n5. **I Gotta Feeling** - The Black Eyed Peas\n6. **Shut Up and Dance** - WALK THE MOON\n7. **Get Lucky** - Daft Punk (feat. Pharrell Williams)\n8. **This Is What You Came For** - Calvin Harris (feat. Rihanna)\n9. **Uptown Funk** - Mark Ronson (feat. Bruno Mars)\n10. **Despacito (Remix)** - Luis Fonsi & Daddy Yankee (feat. Justin Bieber)\n11. **Ritmo (Bad Boys For Life)** - Black Eyed Peas & J

In [16]:
response.choices[0].message.tool_calls[0].id

'call_i8nDLox2SMnNRmaiTJcbzJ0d'

In [39]:
response_playlist = create_spotify_songs("Create a playlist with theme 'a cold winter morning'")
response_playlist

ChatCompletion(id='chatcmpl-CRP6FYnn4DmkVp5noLNEoUig4qZij', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n    "name": "A Cold Winter Morning",\n    "song1": {\n        "artist": "Iron & Wine",\n        "song_name": "Naked as We Came"\n    },\n    "song2": {\n        "artist": "Sufjan Stevens",\n        "song_name": "Chicago"\n    },\n    "song3": {\n        "artist": "Fleet Foxes",\n        "song_name": "White Winter Hymnal"\n    },\n    "song4": {\n        "artist": "The Lumineers",\n        "song_name": "Ho Hey"\n    },\n    "song5": {\n        "artist": "Bon Iver",\n        "song_name": "Holocene"\n    },\n    "song6": {\n        "artist": "Coldplay",\n        "song_name": "Christmas Lights"\n    },\n    "song7": {\n        "artist": "Simon & Garfunkel",\n        "song_name": "The Sound of Silence"\n    },\n    "song8": {\n        "artist": "Ben Howard",\n        "song_name": "Only Love"\n    },\n    "song9": {\n        "arti